In [1]:
from concurrent.futures import ThreadPoolExecutor
from typing import List
from lighthouse_utils import paralelization

# types
import concurrent.futures

In [2]:
if __name__ == "__main__":
    chunk_size = 10  # Adjust chunk size as needed
    file_path = "./WebScreenshots.csv"

    with ThreadPoolExecutor(
        max_workers=10
    ) as executor:  # Adjust number of workers as needed
        futures: List[concurrent.futures._base.Future] = []
        start_id = 0
        for chunk in paralelization.chunked_csv_reader(file_path, chunk_size):
            futures.append(executor.submit(paralelization.process_chunk, chunk, start_id))
            start_id += len(chunk)

        for future in futures:
            future.result()  # Wait for all futures to complete

https://www.primevideo.com/ 

Report saved for lighthouse_report.json
Lighthouse Performance Metrics:
Performance score: 59.0
Accessibility score: 86.0
Best-practices score: 82.0
Seo score: 100
https://www.netflix.com/ 

Error running Lighthouse command for https://www.netflix.com/: Runtime error encountered: Protocol error (Runtime.evaluate): Protocol error (Runtime.evaluate): Target closed
Error: Protocol error (Runtime.evaluate): Protocol error (Runtime.evaluate): Target closed
    at LighthouseError.fromProtocolMessage (file:///c:/Users/cm_cj/Desktop/Tec/aplicaciones/reto/webscraping/Lighthouse/node_modules/lighthouse/core/lib/lh-error.js:160:19)
    at file:///c:/Users/cm_cj/Desktop/Tec/aplicaciones/reto/webscraping/Lighthouse/node_modules/lighthouse/core/gather/session.js:115:29
    at runNextTicks (node:internal/process/task_queues:60:5)
    at process.processImmediate (node:internal/timers:449:9)
    at async ExecutionContext._evaluateInContext (file:///c:/Users/cm_cj/Desktop/T